In [26]:
# Import OANDA config with personal API key and account ID
import config
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions

# Import other necessary libraries
import pandas as pd
import pandas_ta as ta

In [27]:
# Setup connection to OANDA account
client = API(access_token=config.OANDA_API_KEY)

In [28]:
# Define Setup
timeframe = "M5"            # 5 minute candles
count = 500                 # Number of candles
instrument = "SPX500_USD"   # S&P500 CFD

In [29]:
def get_candles(instrument, timeframe, count, client):
    params = {
        "granularity": timeframe,
        "price": "BA",
        "count": count
    }

    r = instruments.InstrumentsCandles(
        instrument=instrument,
        params=params
    )

    client.request(r)
    candles = r.response["candles"]

    data = []

    for c in candles:
        if c["complete"]:
            data.append({
                "time": c["time"],
                "ask_open": float(c["ask"]["o"]),
                "ask_high": float(c["ask"]["h"]),
                "ask_low": float(c["ask"]["l"]),
                "ask_close": float(c["ask"]["c"]),
                "bid_open": float(c["bid"]["o"]),
                "bid_high": float(c["bid"]["h"]),
                "bid_low": float(c["bid"]["l"]),
                "bid_close": float(c["bid"]["c"])
            })

    if not data:
        raise ValueError("No completed candles returned")

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    df.set_index("time", inplace=True)

    return df


In [30]:
def calculate_indicators(df):
    df = df.copy()

    # Mid prices
    df["mid_close"] = (df["ask_close"] + df["bid_close"]) / 2
    df["mid_high"] = (df["ask_high"] + df["bid_high"]) / 2
    df["mid_low"] = (df["ask_low"] + df["bid_low"]) / 2

    # VWAP
    typical_price = (df["mid_high"] + df["mid_low"] + df["mid_close"]) / 3

    df["vwap"] = (
        typical_price.groupby(df.index.date)
        .apply(lambda x: x.cumsum() / (range(1, len(x) + 1)))
        .reset_index(level=0, drop=True)
    )

    # EMA
    df["ema_20"] = ta.ema(df["mid_close"], length=20)

    # Session high / low
    df["session_high"] = df.groupby(df.index.date)["mid_high"].cummax()
    df["session_low"] = df.groupby(df.index.date)["mid_low"].cummin()

    # Clean NaNs
    df = df.dropna(subset=["vwap", "ema_20"])

    return df


In [31]:
def has_open_position(instrument, client):
    r = positions.OpenPositions(config.OANDA_ACCOUNT_ID)
    client.request(r)

    for pos in r.response.get("positions", []):
        if pos["instrument"] == instrument:
            long_units = int(pos["long"]["units"])
            short_units = int(pos["short"]["units"])

            if long_units != 0 or short_units != 0:
                return True

    return False


In [32]:
def place_order(direction, stop_loss, take_profit):
    """
    direction: "long" or "short"
    stop_loss: float
    take_profit: float
    """

    units = 1 if direction == "long" else -1

    data = {
        "order": {
            "instrument": instrument,
            "units": str(units),
            "type": "MARKET",
            "timeInForce": "FOK",

            "stopLossOnFill": {
                "price": f"{stop_loss:.3f}"
            },

            "takeProfitOnFill": {
                "price": f"{take_profit:.3f}"
            }
        }
    }
    r = orders.OrderCreate(
        config.OANDA_ACCOUNT_ID,
        data=data
    )

    client.request(r)

    print(
        f"Placed order for {instrument} "
        f"with stop loss at {round(stop_loss, 3)} "
        f"and take profit at {round(take_profit, 3)}"
    )



In [33]:
def vwap_pullback_trade(df):
    tp_ratio = 2.0  # Risk : Reward

    last_candle = df.iloc[-1]
    previous_candle = df.iloc[-2]

    # ----------------------------
    # LONG SETUP
    # ----------------------------
    long_signal = (
        last_candle["mid_close"] > last_candle["vwap"] and
        last_candle["mid_close"] > last_candle["ema_20"] and
        previous_candle["mid_close"] < previous_candle["ema_20"]
    )

    # ----------------------------
    # SHORT SETUP
    # ----------------------------
    short_signal = (
        last_candle["mid_close"] < last_candle["vwap"] and
        last_candle["mid_close"] < last_candle["ema_20"] and
        previous_candle["mid_close"] > previous_candle["ema_20"]
    )

    if long_signal or short_signal:

        # 🚨 POSITION CHECK (critical)
        if has_open_position(instrument, client):
            print("Position already open — skipping trade")
            return

    if long_signal:
        print("Buy Signal: VWAP pullback continuation")

        entry_price = last_candle["ask_close"]
        stop_loss = df["bid_low"].iloc[-3:-1].min()

        risk = entry_price - stop_loss
        take_profit = entry_price + tp_ratio * risk

        if risk <= 0:
            print("Invalid risk — skipping trade")
            return


        place_order(
            direction="long",
            stop_loss=stop_loss,
            take_profit=take_profit
        )

    elif short_signal:
        print("Sell Signal: VWAP pullback continuation")

        entry_price = last_candle["bid_close"]
        stop_loss = df["ask_high"].iloc[-3:-1].max()

        risk = stop_loss - entry_price
        take_profit = entry_price - tp_ratio * risk

        if risk <= 0:
            print("Invalid risk — skipping trade")
            return


        place_order(
            direction="short",
            stop_loss=stop_loss,
            take_profit=take_profit
        )

    else:
        print("Strategy conditions not met")


In [34]:
def run_bot():
    print("Starting trading bot")

    price = get_candles(instrument, timeframe, count, client)
    price = calculate_indicators(price)

    vwap_pullback_trade(price)

In [35]:
run_bot()

Starting trading bot
Strategy conditions not met
